주식 백테스트

In [112]:
from pykrx import stock
import numpy as np
import pandas as pd
import time

In [ ]:
#
start_year = "2015"
start_date = start_year+"0101"
end_date = str(int(start_date[:4])+1) + "1231"
target = '1028'
original_df = stock.get_index_ohlcv(start_date, end_date, target)

In [103]:
start_year = "2017"
start_date = start_year+"0101"
end_date = str(int(start_date[:4])+1) + "1231"
target = '1028'
original_df2 = stock.get_index_ohlcv(start_date, end_date, target)

In [115]:
merged_df = pd.DataFrame()
for i in range(2000,2023):
    start_date = str(i)+"0101"
    end_date = str(i) + "1231"
    target = '1001'
    res_df = stock.get_index_ohlcv(start_date, end_date, target)
    time.sleep(1)
    merged_df = pd.concat([merged_df, res_df])
merged_df

코스피,시가,고가,저가,종가,거래량,거래대금,상장시가총액
날짜,,,,,,,
2000-01-04,1028.33,1066.18,1016.59,1059.04,195898660,3771013000000,0
2000-01-05,1006.87,1047.70,984.05,986.31,257696170,4798126000000,0
2000-01-06,1013.95,1014.90,953.50,960.79,203523570,3726898000000,0
2000-01-07,949.17,970.16,930.84,948.65,215664150,3696686000000,0
2000-01-10,979.67,994.94,965.02,987.24,240175480,3815037000000,0
...,...,...,...,...,...,...,...
2022-12-23,2325.86,2333.08,2311.90,2313.69,366988585,5935798139102,1829395536163727
2022-12-26,2312.54,2321.92,2304.20,2317.14,427844636,5205750204132,1831727639591539
2022-12-27,2327.52,2335.99,2321.48,2332.79,448498792,7126994120167,1844141209320557


In [ ]:
merged_df

In [116]:
ma_period = 5

df = merged_df[['종가']].copy()
ma = df['종가'].rolling(ma_period).mean()
df['ma'] = ma.shift(1)

cond = df['종가'] > df['ma']
df['status'] = np.where(cond, 1, 0)
df.iloc[-1, -1] = 0

매수조건 = (df['status']==1) & (df['status'].shift(1)==0)
매도조건 = (df['status']==0) & (df['status'].shift(1)==1)
df['bs'] = np.where(매수조건, 1, 0)
df['ss'] = np.where(매도조건, 1, 0)

slippage = 0.005
수익률 = df.loc[매도조건, '종가'].reset_index(drop=True) / df.loc[매수조건, '종가'].reset_index(drop=True)
수익률.apply(lambda x:x-slippage)
print(수익률.cumprod())

0      0.954753
1      0.966463
2      0.951868
3      0.901509
4      0.891362
         ...   
621    4.024756
622    4.026311
623    3.962071
624    3.889713
625    3.802441
Name: 종가, Length: 626, dtype: float64


C:\Users\jchoi\AppData\Local\Temp\ipykernel_38124\3132413612.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['ma'] = ma.shift(1)
C:\Users\jchoi\AppData\Local\Temp\ipykernel_38124\3132413612.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['status'] = np.where(cond, 1, 0)
C:\Users\jchoi\AppData\Local\Temp\ipykernel_38124\3132413612.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the ca